In [4]:
import simplekml
import sqlite3
import pandas as pd

In [5]:
grouplist = ['No CRB damage','Old damage','New damage','New and old damage','Dead standing palm']
criterialist = ['ecplus_c11 IS NULL OR ecplus_c11 LIKE "no CRB damage"',
                'ecplus_c11 LIKE "old damage" OR ecplus_c11 LIKE "old damage,bore hole" OR ecplus_c11 LIKE "bore hole"',
                'ecplus_c11 LIKE "new damage" OR ecplus_c11 LIKE "new damage,bore hole"',
                'ecplus_c11 LIKE "new damage,old damage,bore hole" OR ecplus_c11 LIKE "new damage,old damage"',
                'ecplus_c11 LIKE "standing dead palm (snag)"']
green = 0xFF00FF14
yellow = 0xFF14F0FF
orange = 0xFF1478FF
red = 0xFF1400FF
black = 0xFF000000
colorlist = [green, yellow, orange, red, black]
d = {'group':grouplist, 'criteria':criterialist, 'color':colorlist}
df = pd.DataFrame(d, columns=('group','criteria', 'color'))
df

,group,criteria,color
0,No CRB damage,"ecplus_c11 IS NULL OR ecplus_c11 LIKE ""no CRB ...",4278255380
1,Old damage,"ecplus_c11 LIKE ""old damage"" OR ecplus_c11 LIK...",4279562495
2,New damage,"ecplus_c11 LIKE ""new damage"" OR ecplus_c11 LIK...",4279531775
3,New and old damage,"ecplus_c11 LIKE ""new damage,old damage,bore ho...",4279501055
4,Dead standing palm,"ecplus_c11 LIKE ""standing dead palm (snag)""",4278190080


In [6]:
kml = simplekml.Kml()
#con = sqlite3.connect('AAFB-CRB-Damage.sqlite')
con = sqlite3.connect('navy-trees.sqlite')
for index, row in df.iterrows():
    cursor = con.cursor()
    sql = '''
    SELECT ecplus_c_2 as latitude, ecplus_c_3 as longitude 
    FROM navy_trees
    WHERE {}
    '''.format(row.criteria)
    cursor.execute(sql)
    fol = kml.newfolder(name=row.group)
    for r in cursor.fetchall():
        latitude, longitude = r
        pnt = fol.newpoint(coords=[(longitude, latitude)])
        pnt.style.iconstyle.color = hex(row.color)
    cursor.close
kml.savekmz("navy-crb-damage.kmz")